<a href="https://colab.research.google.com/github/azka2001/dip-proj/blob/main/DIPProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
!pip install darkflow.net.build
from darkflow.net.build import TFNet
import cv2
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement darkflow.net.build (from versions: none)
ERROR: No matching distribution found for darkflow.net.build


ModuleNotFoundError: ignored

In [11]:
# Install required libraries
!pip install tensorflow-gpu==1.15.0rc2
!pip install imageio

# Download and build darkflow (the tensorflow implementation of YOLO)
import os
import pathlib

if "darkflow-master" in pathlib.Path.cwd().parts:
  while "darkflow-master" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path("darkflow-master").exists():
  !git clone --depth 1 https://github.com/thtrieu/darkflow.git
  # Compile darkflow
  %cd darkflow
  !python setup.py build_ext --inplace
  # Change darkflow to darkflow-master to distinguish between folder names
  %cd ../
  !mv darkflow darkflow-master
  %cd darkflow-master

# Upload yolo.weights, pre-trained weights file (for YOLO v2) from an external Google drive 
weights = 'yolo'
weights_file = weights + '.weights'
if not os.path.exists('weights_file'):
  !gdown --id 0B1tW_VtY7oniTnBYYWdqSHNGSUU
  !mkdir bin
  !mv yolo.weights bin

# Imports
%cd darkflow-master
%tensorflow_version 1.15.0rc2

# For importing/exporting files, working with arrays, etc
import time
import urllib
import numpy as np
import pandas as pd
import imageio

# For actual object detection
import tensorflow as tf
from darkflow.net.build import TFNet
threshold = 0.25

# For drawing onto and plotting images
import matplotlib.pyplot as plt
import cv2
%config InlineBackend.figure_format = 'svg'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.15.0rc2 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==1.15.0rc2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'darkflow'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 105 (delta 21), reused 99 (delta 21), pack-reused 0
Receiving objects: 100% (105/105), 16.41 MiB | 8.99 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/darkflow
/content/darkflow/setup.py:6: DeprecationWarning: the im

ValueError: ignored

In [6]:
# Load image
image = cv2.imread("/content/sample_data/content/sample_data/Red_Frames/frame1013.jpg")


# Convert image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define thresholds for red cards in HSV color space
red_min1 = (0, 100, 100)
red_max1 = (10, 255, 255)
red_min2 = (160, 100, 100)
red_max2 = (179, 255, 255)

# Threshold the image to get the red card mask
red_mask1 = cv2.inRange(hsv, red_min1, red_max1)
red_mask2 = cv2.inRange(hsv, red_min2, red_max2)
red_mask = cv2.bitwise_or(red_mask1, red_mask2)

# Find contours in the mask
contours, hierarchy = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter out contours that are too small or too large to be cards
min_area = 200
max_area = 10000
cards = []
for contour in contours:
    area = cv2.contourArea(contour)
    if area > min_area and area < max_area:
        cards.append(contour)

# Draw rectangles around the red cards
for card in cards:
    (x, y, w, h) = cv2.boundingRect(card)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

# Save annotated image
cv2.imwrite("annotated_rframe06.jpg", image)

True

In [ ]:
# Load image
image = cv2.imread("/content/sample_data/Yellow Frames/frame17.jpg")


# Convert image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define thresholds for yellow cards in HSV color space
yellow_min1 = (20, 100, 100)
yellow_max1 = (30, 255, 255)
yellow_min2 = (50, 100, 100)
yellow_max2 = (60, 255, 255)

# Threshold the image to get the yellow card mask
yellow_mask1 = cv2.inRange(hsv, yellow_min1, yellow_max1)
yellow_mask2 = cv2.inRange(hsv, yellow_min2, yellow_max2)
yellow_mask = cv2.bitwise_or(yellow_mask1, yellow_mask2)

# Find contours in the mask
contours, hierarchy = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter out contours that are too small or too large to be cards
min_area = 150
max_area = 2000
cards = []
for contour in contours:
    area = cv2.contourArea(contour)
    if area > min_area and area < max_area:
        cards.append(contour)
    
    # Draw rectangles around the yellow cards
for card in cards:
    (x, y, w, h) = cv2.boundingRect(card)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)


# Save the annotated image
cv2.imwrite("annotated_iframe02.jpg", image)

True

In [ ]:
# Load image
image = cv2.imread("/content/sample_data/yellowcard detection/imagey04.jpg")


# Convert image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define thresholds for yellow cards in HSV color space
yellow_min1 = (20, 100, 100)
yellow_max1 = (30, 255, 255)
yellow_min2 = (50, 100, 100)
yellow_max2 = (60, 255, 255)

# Threshold the image to get the yellow card mask
yellow_mask1 = cv2.inRange(hsv, yellow_min1, yellow_max1)
yellow_mask2 = cv2.inRange(hsv, yellow_min2, yellow_max2)
yellow_mask = cv2.bitwise_or(yellow_mask1, yellow_mask2)

# Find contours in the mask
contours, hierarchy = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter out contours that are too small or too large to be cards
min_area = 10
max_area = 40
cards = []
for contour in contours:
    area = cv2.contourArea(contour)
    if area > min_area and area < max_area:
        cards.append(contour)
    
    # Draw rectangles around the yellow cards
for card in cards:
    (x, y, w, h) = cv2.boundingRect(card)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)


# Save the annotated image
cv2.imwrite("annotated_imagey04.jpg", image)

In [ ]:
# Read the video from specified path
cam = cv2.VideoCapture("/content/Gabriel Martinelli’s two yellow cards in five seconds (1).mp4")
  
try:
      
    # creating a folder named data
    if not os.path.exists('/content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)'):
        os.makedirs('/content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)')
  
# if not created then raise error
except OSError:
    print ('Error: Creating directory of data')
  
# frame
currentframe = 0
  
while(True):
      
    # reading from frame
    ret,frame = cam.read()
  
    if ret:
        # if video is still left continue creating images
        # save frame
        name = '/content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name)
  
        # writing the extracted images
        cv2.imwrite(name, frame)
  
        # increasing counter so that it will
        # show how many frames are created
        currentframe += 10 # i.e. at 30 fps, this advances one second
        cam.set(1, currentframe)
    else:
        break

cam.release()
cv2.destroyAllWindows()

Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame0.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame10.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame20.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame30.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame40.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame50.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame60.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame70.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five seconds (1)/frame80.jpg
Creating.../content/sample_data/Gabriel Martinelli’s two yellow cards in five secon

In [ ]:
options = {"model": "/content/yolo_custom.cfg", 
           "load": "bin/yolo.weights",
           "batch": 8,
           "epoch": 100,
           "gpu": 1.0,
           "train": True,
           "annotation": "./annotations/",
           "dataset": "./images/"}

In [ ]:
tfnet = TFNet(options)

In [ ]:
tfnet.train()

In [ ]:
options = {"model": "cfg/yolo_custom.cfg",
           "load": -1,
           "gpu": 1.0}

In [ ]:
tfnet2 = TFNet(options)

In [ ]:
tfnet2.load_from_ckpt()

In [ ]:
import pprint as pp

original_img = cv2.imread("/content/sample.jpg")
original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
results = tfnet2.return_predict(original_img)
print(results)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(original_img)

In [ ]:
def boxing(original_img , predictions):
    newImage = np.copy(original_img)

    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))
        
        if confidence > 0.3:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
        
    return newImage

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(boxing(original_img, results))

In [ ]:
from math import ceil

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))

for i in range(5):
    original_img = cv2.imread("/content/sample" + str(i+1) + ".jpg")
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    results = tfnet2.return_predict(original_img)
    
    ax[ceil(i/3)-1, i%3].imshow(boxing(original_img, results))

In [ ]:
cap = cv2.VideoCapture('./sample_video/test_video.avi')
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) 

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('./sample_video/output.avi',fourcc, 20.0, (int(width), int(height)))

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if ret == True:
        frame = np.asarray(frame)      
        results = tfnet2.return_predict(frame)
        
        new_frame = boxing(frame, results)

        # Display the resulting frame
        out.write(new_frame)
        cv2.imshow('frame', new_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('xyz')